### Transform Data to be a suitable Input to RNNS

In [ ]:
# Modifify 1-dimensional age-data to be a suitable RNN input
def rnn_age_transform_data(data, n,scale_up, scale_low =0, replication_type = 'repetitive'):
    data_transformed = []
    for i in range(len(data)):
        if replication_type == 'adaptive': data_transformed.append(np.int_(np.linspace(data[i], data[i]+n, num =n+1))) # Version 1: Update Current Age
        if replication_type == 'repetitive': data_transformed.append(np.repeat(2*(data[i]-age_low)/(age_up-age_low)-1,n+1)) # Version 2: Repetitive Input of Age

    return np.asarray(data_transformed).reshape(len(data),n+1,1)

In [1]:
# transform 4-dim feature data to RNN suitable input
# This is a special case of the more general data_full_transform_test function.

def data_full_transform(data, n_aim):
    
    n_features = data.shape[1]
    n_len = data.shape[0]
    data_new = []

    # Aim: Transform n_len x n_features matrix to n_len x n_aim (=dur_max+1 or user-input) x n_features 3D-array
    
    for i in range(n_len):
        step1 = np.tile(data[i,],(min(max(data[i,2]-data[i,3],0), n_aim),1)) # repeat input for rem. duration (limited by user input)
        step2 = np.tile(np.repeat(-1,n_features),(max(n_aim-max(data[i,2]-data[i,3],0),0),1)) # fill up remaining length by -1's
        data_new.append(np.concatenate((step1,step2)))
        
    return np.asarray(data_new).reshape(n_len,n_aim, n_features)

In [6]:
# transform 4-dim feature data to RNN suitable input
# input_type: 'raw' data or 'scaled' data
# If input_type = 'scaled': We need Max_min to rescale duration and age_of_contract to know how often to repeat the input

def data_full_transform_test(data, n_aim, input_type = 'raw', Max_min = None):
    
    n_features = data.shape[1]
    n_len = data.shape[0]
    data_new = []

    # Aim: Transform n_len x n_features matrix to n_len x n_aim (=dur_max+1 or user-input) x n_features 3D-array
    
    for i in range(n_len):
        if input_type == 'raw': # alias known as adjusted input
            # repeat input for rem. duration (limited by user input)
            step1 = np.tile(data[i,],(min(data[i,2]-data[i,3], n_aim),1)) 
            # fill up remaining length by -1's
            step2 = np.tile(np.repeat(-1,n_features),(max(n_aim-data[i,2]+data[i,3],0),1)) 
        elif input_type == 'scaled':
            dur = (data[i,2]+1)/2*(Max_min[2,1]-Max_min[2,0])+Max_min[2,0]
            aoc = (data[i,3]+1)/2*(Max_min[3,1]-Max_min[3,0])+Max_min[3,0]
            # repeat input for rem. duration (limited by user input)
            step1 = np.tile(data[i,],(min(int(dur-aoc), n_aim),1)) 
            # fill up remaining length by -1's
            step2 = np.tile(np.repeat(-1,n_features),(max(n_aim-int(dur-aoc),0),1))
        elif input_type == 'plain':
            step1 = np.tile(data[i,],n_aim)
            step2 = None
        else:
            print('Unknown input_type.')
            pass
        if input_type == 'plain':
            data_new.append(step1)
        else:
            data_new.append(np.concatenate((step1,step2)))
        
    return np.asarray(data_new).reshape(n_len,n_aim, n_features)